<a href="https://colab.research.google.com/github/thejeswar2419/StudentGitLabRepo/blob/main/chapter_appendix-tools-for-deep-learning/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# ============================
# 1. Mount Google Drive
# ============================
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
import shutil

# ============================
# 2. CONFIG: Set your folders in Drive
# ============================
# Folder in Drive that contains your input screenshots
# Example: /content/drive/MyDrive/ui_screenshots
INPUT_DIR = "/content/drive/MyDrive/deepui/dataset/test/sheets"

# Folder in Drive where annotated images will be saved
OUTPUT_DIR = "/content/drive/MyDrive/deepui/Annotated_dataset/test/sheets"

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📂 Reading images from: {INPUT_DIR}")
print(f"💾 Saving annotated images to: {OUTPUT_DIR}")

# ============================
# 3. COLOR MAPS FOR UI CLASSES
# ============================
COLOR = {
    "text":   (255, 0, 0),      # Blue
    "button": (0, 255, 0),      # Green
    "icon":   (0, 255, 255),    # Yellow
    "image":  (255, 0, 255)     # Purple
}

# ============================
# 4. CLASSIFICATION LOGIC
# ============================
def classify_box(w, h):
    aspect = w / (h + 1e-6)
    area = w * h

    # TEXT → long & thin
    if h < 45 and aspect > 2:
        return "text"

    # BUTTON → medium rectangle
    if 45 <= h <= 120 and 1.2 < aspect < 6:
        return "button"

    # ICON → small square-ish
    if area < 18000 and 0.7 < aspect < 1.4:
        return "icon"

    # IMAGE → large square-ish block
    if 0.7 < aspect < 1.4:
        return "image"

    # fallback
    return "button"

# ============================
# 5. ANNOTATION FUNCTION
# ============================
def annotate_image(img_path, save_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠ Could not read: {img_path}")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dil = cv2.dilate(edges, kernel, iterations=2)

    contours,_ = cv2.findContours(dil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    annotated = img.copy()

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Ignore noise
        if w < 30 or h < 20 or w*h < 600:
            continue

        # Expand bounding box
        pad = 5
        x1 = max(0, x - pad)
        y1 = max(0, y - pad)
        x2 = min(img.shape[1], x + w + pad)
        y2 = min(img.shape[0], y + h + pad)

        cls = classify_box(w, h)
        color = COLOR[cls]

        # Draw box
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)

        # Draw label background
        cv2.rectangle(annotated, (x1, y1 - 22), (x1 + 130, y1), color, -1)

        # Draw label text
        cv2.putText(annotated, cls, (x1 + 4, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2)

    cv2.imwrite(save_path, annotated)

# ============================
# 6. PROCESS ALL IMAGES FROM DRIVE FOLDER
# ============================
count = 0
for root, dirs, files in os.walk(INPUT_DIR):
    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            in_path = os.path.join(root, f)
            out_path = os.path.join(OUTPUT_DIR, f)
            annotate_image(in_path, out_path)
            count += 1

print(f"✔ Annotated {count} images → saved in {OUTPUT_DIR}")

# ============================
# 7. (OPTIONAL) ZIP THE OUTPUT FOLDER IN DRIVE
# ============================
# This will create a ZIP inside Drive, not download it
#"""zip_base = os.path.join("/content/drive/MyDrive", "annotated_labeled_zip")  # no .zip here
#shutil.make_archive(zip_base, "zip", OUTPUT_DIR)

#print(f"📦 ZIP created at: {zip_base}.zip")"""


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Reading images from: /content/drive/MyDrive/deepui/dataset/test/sheets
💾 Saving annotated images to: /content/drive/MyDrive/deepui/Annotated_dataset/test/sheets
✔ Annotated 45 images → saved in /content/drive/MyDrive/deepui/Annotated_dataset/test/sheets


In [ ]:
import os

INPUT_DIR = "/content/drive/MyDrive/DeepUI/dataset/test/angelone"

print("Listing contents of:", INPUT_DIR)
for root, dirs, files in os.walk(INPUT_DIR):
    print("📁 Folder:", root)
    for f in files:
        print("   -", f)


Listing contents of: /content/drive/MyDrive/DeepUI/dataset/test/angelone


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!find "/content/drive/MyDrive/deepui" -maxdepth 5 -type d



find: ‘/content/drive/MyDrive/deepui’: No such file or directory
